<a href="https://colab.research.google.com/github/Tushar1344/dl_from_scratch/blob/main/Gpt2_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken >> //dev/null
!pip install matplotlib >> //dev/null





# Read in GPT2 using Huggingface

In [2]:
from transformers import GPT2LMHeadModel

In [3]:
# model_hf = GPT2LMHeadModel.from_pretrained("gpt2")
# sd_hf = model_hf.state_dict()

# for k,v in sd_hf.items():
#   print(k,v.shape)

In [4]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.imshow(sd_hf["transformer.wpe.weight"])

In [5]:
#plt.plot(sd_hf["transformer.wpe.weight"][:, 150])
#plt.plot(sd_hf["transformer.wpe.weight"][:, 200])
#plt.plot(sd_hf["transformer.wpe.weight"][:, 250])

In [6]:
# plt.imshow(sd_hf["transformer.h.1.attn.c_attn.weight"][:300, :300],cmap = "gray")

In [7]:
# from transformers import pipeline, set_seed

# generator = pipeline('text-generation', model='gpt2')
# set_seed(42)
# generator("Hello, I'm a language model", max_length=30, num_return_sequences=5)

# Building your own GPT2

In [8]:
from dataclasses import dataclass  #for building the config object
import torch
import math
import torch.nn as nn
from torch.nn import functional as F

In [9]:
@dataclass
class GPTConfig:
  block_size: int = 1024 #Context length
  vocab_size: int = 50257 #Number of tokens
  n_layer: int = 12 #Number of layers
  n_head: int = 12  #Number of heads
  n_embd: int = 768 #Embedding size


In [10]:
class CausalSelfAttention(nn.Module):
  '''
  A vanilla multi-head masked self-attention layer with a projection at the end
  '''
  def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # Key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # Output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # Regularization not present
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        # Not really a "bias", more of a mask, but following the OpenAI/HF naming though
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                             .view(1, 1, config.block_size, config.block_size))

  def forward(self, x):
        B, T, C = x.size()  # Batch size, sequence length, embedding dimensionality (n_embd)
        # Calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        # Causal Self-Attention
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v  # (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C)  # Re-assemble all head outputs side by side
        y = self.c_proj(y)
        return y


In [11]:
csa_obj = CausalSelfAttention(GPTConfig())

In [12]:
class MLP(nn.Module):
  '''MLP layer with GELU activation'''
  def __init__(self, config):
    super().__init__()
    self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
    self.gelu = nn.GELU(approximate= 'tanh')
    self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)

  def forward(self, x):
    x = self.c_fc(x)
    x = self.gelu(x)
    x = self.c_proj(x)
    return x

In [13]:
mlp_obj = MLP(GPTConfig())

In [14]:
class Block(nn.Module):
  '''Transformer block: communication followed by computation'''
  def __init__(self, config):
      super().__init__()
      self.ln_1 = nn.LayerNorm(config.n_embd)
      self.attn = CausalSelfAttention(config)
      self.ln_2 = nn.LayerNorm(config.n_embd)
      self.mlp = MLP(config)

  def forward(self, x):
    x = x + self.attn(self.ln_1(x))
    x = x + self.mlp(x)
    return x

In [15]:
block_obj = Block(GPTConfig())

# GPT2 loading class

In [16]:
#class for loading gpt2 weights without having to use the huggingface library

import torch.nn as nn
import torch
from transformers import GPT2LMHeadModel

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

    def forward(self, idx, targets = None):
      # idx is of shape (B, T)
      B, T = idx.size()
      assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is {self.config.block_size}"

      # Forward the token and position embeddings
      pos = torch.arange(0, T, dtype=torch.long, device=idx.device)  # shape (T)
      pos_emb = self.transformer.wpe(pos)  # position embeddings of shape (T, n_embd)
      tok_emb = self.transformer.wte(idx)  # token embeddings of shape (B, T, n_embd)
      x = tok_emb + pos_emb

      # Forward the blocks of the transformer
      for block in self.transformer.h:
          x = block(x)

      # Forward the final layernorm and the classifier
      x = self.transformer.ln_f(x)
      logits = self.lm_head(x)  # (B, T, vocab_size)
      loss = None
      if targets is not None:
          loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))


      return logits, loss


    @classmethod
    def from_pretrained(cls, model_type):
        """
        Loads pretrained GPT-2 model weights from Huggingface
        """
        assert model_type in ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']

        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = dict(
            gpt2=dict(n_layer=12, n_head=12, n_embd=768),   # 124M params
            gpt2_medium=dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            gpt2_large=dict(n_layer=36, n_head=20, n_embd=1280),  # 774M params
            gpt2_xl=dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        )[model_type]

        config_args['vocab_size'] = 50257  # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024  # always 1024 for GPT model checkpoints

        # Create a from-scratch initialized miniGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)

        # Load state_dict from pretrained model
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')]  # discard this mask / buffer

        # Initialize a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # Copy while ensuring all the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]  # ignore these
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')]  # same, just the mask

        # Transposed keys for special handling
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

        assert len(sd_keys_hf) == len(sd_keys), "Mismatched keys: {} vs {}".format(len(sd_keys_hf), len(sd_keys))

        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # Special treatment for the Conv1D weights that need to be transposed
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # Vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        model.load_state_dict(sd)
        return model

# Block and GPTConfig would need to be defined elsewhere in the code


In [17]:
#model = GPT.from_pretrained('gpt2')

In [18]:
# #tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# with open('input.txt', 'r', encoding='utf-8') as f:
#     text = f.read()
# data = text[:1000]
# print(data[:100])

In [19]:
# !wc input.txt

# Initial Train.py

In [20]:
%tb
#Train.py code
#attempt to autodetect the device
import torch
import tiktoken
from torch.optim import AdamW


device = "cpu"
if torch.cuda.is_available():
  device = "cuda"
elif torch.backends.mps.is_available():
  device = "mps"
print(f"using device: {device}")

#device ="cpu" #OVERRIDE

#tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
data = text[:1000]
print(data[:100])

#get a data batch

enc = tiktoken.get_encoding("gpt2")
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
text = text[:1000]
tokens = enc.encode(text)
B, T = 4, 32
buf = torch.tensor(tokens[:B*T+1])
buf = buf.to(device)
x = buf[:-1].view(B, T)
y=buf[1:].view(B, T)

#get logits
config = GPTConfig()
model = GPT(config)
model.to(device)
#x = x.to(device)
#y = y.to(device)
#logits, loss = model(x, y)

#optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for i in range(50):
  logits, loss = model(x, y)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  print(f"step{i}, loss: {loss.item()}")
#print(logits.shape)
#print(loss)

using device: cuda
--2024-09-09 02:57:30--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2024-09-09 02:57:30 (107 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



No traceback available to show.


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
step0, loss: 10.97608470916748
step1, loss: 6.763216495513916
step2, loss: 6.228250980377197
step3, loss: 3.105747938156128
step4, loss: 1.9845982789993286
step5, loss: 1.1414642333984375
step6, loss: 0.6535788774490356
step7, loss: 0.40266725420951843
step8, loss: 0.2823127210140228
step9, loss: 0.1855582892894745
step10, loss: 0.12641243636608124
step11, loss: 0.09013315290212631
step12, loss: 0.06664373725652695
step13, loss: 0.05083441734313965
step14, loss: 0.04002544283866882
step15, loss: 0.03346583992242813
step16, loss: 0.027835454791784286
step17, loss: 0.023455502465367317
step18, loss: 0.019874952733516693
step19, loss: 0.01707400009036064
step20, loss: 0.014949685893952847
step21, loss: 0.013048483058810234
step22, loss: 0.011447412893176079
step23, loss: 0.010210562497377396
step24, loss: 0.009247585199773312
step25, loss: 0.00845411978662014
step26, loss: 0.00776235666126

In [21]:
import tiktoken

class DataloaderLite:
  def __init__(self, B, T):
    self.B = B
    self.T = T

    #at init load tokens from disk and store them in memory
    with open('input.txt', 'r', encoding='utf-8') as f:
      text = f.read()
    enc = tiktoken.get_encoding("gpt2")
    tokens = enc.encode(text)
    self.tokens = torch.tensors(tokens)
    print(f"loaded {len(tokens)} tokens")
    print(f"1 epoch of data is {len(tokens)//(B*T)} batches")

    #state
    self.current_position = 0

  def next_batch(self):
    B, T = self.B, self.T
    buf = self.tokens[self.current_position:self.current_position+B*T+1]
    x = buf[:-1].view(B, T) #inputs
    y=buf[1:].view(B, T) #targets
    #advance position in the tensor
    self.current_position += B * T
    #if loading the next batch will be OOB, then reset
    if self.current_position + B * T >= len(self.tokens):
      self.current_position = 0
    return x, y



In [22]:
num_return_sequences = 5
max_length = 30

#model= GPT.from_pretrained('gpt2')
model= GPT(GPTConfig())
model.eval()
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [23]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode(data)
print(tokens[:24])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13]


In [24]:
#creating labels from tokens for GPT2
import torch
buf = torch.tensor(tokens[:24+1])
x = buf[:-1].view(4,6)
y=buf[1:].view(4,6)
print(x)
print(y)

tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])
tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])


In [25]:
#prefix tokens

#gpt roughly has a 3:1 compression ratio for characters to tokens
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model", allowed_special={"<|endoftext|>"})
tokens = torch.tensor(tokens, dtype=torch.long, device='cuda')
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(device)